# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1099511623296, 1, 1099511623296, 1, 1, 1099511623296, 1099511623296, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1099511623296, 0, 1, 1, 1099511623296, 0, 1099511623296, 1099511623296, 1, 1, 1099511623296, 0, 1099511623296, 0, 1, 1099511623296, 1099511623296, 1, 1, 1099511623296, 1, 0, 0, 1, 1099511623296, 1, 1099511623296, 1099511623296, 1, 1099511623296, 1, 1, 1, 1, 0, 0, 0, 1099511623296, 1, 0, 1]]


In [4]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [5]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1]     [1]     [1, 0, 0, 1]


### Decodes back to Integer

In [6]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

1
1
9


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [7]:
encrypter = Encryptor(ctx, pk)

In [8]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [9]:
decrypter = Decryptor(ctx, sk)

In [10]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [11]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

1     1     9


## Evaluator

In [12]:
eval = Evaluator(ctx)

In [13]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

2


In [14]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

2


In [15]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

2


### Verify result

In [16]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [17]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[90979862835, 1052149874892, 38398099035, 700880002771, 898593201206, 681273170635, 884198174502, 335666540504, 219006038581, 723803820947, 944654992253, 129590741525, 144805372829, 291895483343, 1043484675649, 291646129027, 79067129309, 999036565951, 255484132947, 5408417257, 215498146307, 440565260989, 321865633612, 158748997911, 953086709055, 528518474672, 506038193381, 384721722711, 361817151887, 173099647271, 406367772597, 533784475308, 686120981266, 852880015973, 822706788918, 113020698147, 973150471798, 940679336244, 569147484651, 95039604550, 1004704997085, 73845137664, 1079234407474, 980766395864, 986106147323, 650865009575, 682857749796, 914843701407, 444249598275, 763871993020, 838406283506, 1010743825047, 675542354965, 112204032658, 354956906415, 592095637683, 516585826097, 153788493227, 609180315128, 612170358135, 138665341756, 406413528644, 587037733296, 35476242117]], [[786103533508, 1045935361475, 516289190285, 872730636179, 354457471095, 87012680049, 18793277

In [18]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

1      1


In [39]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(128, CoeffModulus().create(64, [30, 30]), 64))

In [40]:
key_generator = KeyGenerator(context)
keys = key_generator.keygen()
relin_key = key_generator.get_relin_keys()

print(relin_key.data)

[[[[453695769, 552930889, 693947014, 674231369, 984351982, 1064425845, 40644867, 462614856, 1019748881, 947852071, 126206589, 414864002, 1009635783, 972298090, 894077716, 634929242, 566380008, 776522442, 623511219, 444019581, 308375216, 307699888, 591457056, 867305169, 457406749, 398698831, 62499734, 889470614, 384511411, 307052309, 538087952, 717416371, 22084996, 919918420, 600280609, 976074662, 856234064, 1018218603, 129810937, 1009462116, 719852135, 359396518, 79722314, 656983599, 446827454, 917638423, 978854444, 179752550, 864401749, 1034846981, 7510516, 663253628, 670367598, 962565306, 29447587, 35915693, 520623830, 985356566, 6051708, 43701912, 976765781, 717079461, 103948104, 815597593, 414667244, 593486504, 909525037, 921067801, 831396938, 417299932, 933700441, 79905604, 728979568, 49084434, 577286368, 495480635, 860498407, 696603025, 124150923, 632009697, 719333405, 365014698, 508309905, 998894013, 929667664, 894148802, 303372541, 31040486, 598849110, 1067437187, 745847319, 22

In [41]:
x = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [42]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12b64b910>

In [43]:
y = x.decrypt(private_key = keys[0])
y

tensor([[1, 2, 3],
        [4, 5, 6]])

In [44]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12b64b910>

In [45]:
x = x + y

In [46]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12d108790>

In [47]:
x.decrypt(private_key = keys[0])

tensor([[ 2,  4,  6],
        [ 8, 10, 12]])

In [48]:
x = x - th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [49]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12c62cd60>

In [50]:
x.decrypt(private_key = keys[0])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [51]:
 x = x*x

In [52]:
x.decrypt(private_key = keys[0])

tensor([[ 1,  4,  9],
        [16, 25, 36]])

In [53]:
a = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)
b = th.Tensor([[-1,-2],[-3,-4], [-5,-6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [54]:
a

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12c97fac0>

In [55]:
b

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12c97fbb0>

In [60]:
ans_mm = a.mm(b, relin_key = relin_key)
print(ans_mm)

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12c895520>, <syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12c752eb0>], [<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12ca0faf0>, <syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12d0effa0>]]


In [62]:
mm_decrypt = ans_mm.decrypt(private_key = keys[0])

AttributeError: 'list' object has no attribute 'flatten'